In [6]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Proyectos/spotify_analisys/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Proyectos/spotify_analisys


In [16]:
# Instalar BeautifulSoup y requests
!pip install requests beautifulsoup4
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 6.4 MB/s eta 0:00:00


In [17]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [10]:
# URL del Top 50 Global de Spotify Charts
url = "https://open.spotify.com/playlist/37i9dQZEVXbO3qyFxbkOE1"

# Realizar la solicitud a la página
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # Aquí extraemos los nombres de las canciones del top 50 de México
    songs = soup.find_all("div", attrs={"aria-label": True})

    # Extraer e imprimir los valores del atributo aria-label
    for idx, song in enumerate(songs[:50], start=1):  # Limitar a las primeras 10 canciones
        print(f"{idx}. {song['aria-label']}")
else:
    print(f"Error al acceder a la página: {response.status_code}")


1. DtMF
2. Loco
3. ROSONES
4. BAILE INoLVIDABLE
5. NADIE
6. Presidente
7. DOS DÍAS
8. ME JALO
9. Tu Boda
10. EoO
11. NUEVAYoL
12. gervonta
13. ROSONES
14. VOY A LLeVARTE PA PR
15. EL LOKERON
16. ESCAPATE
17. Morena Canela
18. WELTiTA
19. 5-7
20. Qué Pasaría...
21. Amapola
22. AY MAMÁ
23. VeLDÁ
24. Khé?
25. ROCKSTAR
26. PERFuMITO NUEVO
27. KLOuFRENS
28. EL CLúB
29. LA PATRULLA
30. Madonna


In [ ]:
'''
Para poder enriquecer los datos que obtuvimos anteriormente necesitamos
autenticarnos en Spotify para poder obtener mas información de las canciones
'''

#Autenticación en la API de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id =
)